In [1]:
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.datastore import Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.environment import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.exceptions import ComputeTargetException
from azureml.data.data_reference import DataReference
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.train.dnn import TensorFlow
# from azureml.train.hyperdrive import *
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

import os
import shutil
import urllib
import numpy as np
# import matplotlib.pyplot as plt

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.3.0


## Initialize workspace

Initialize a workspace object from persisted configuration. If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

avadevitsmlsvc
RG-ITSMLTeam-Dev
westus2
ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e


In [3]:
exp = Experiment(workspace=ws, name='Hyperdrive_crazy')

In [4]:
ds = ws.get_default_datastore()
# we're assuming that the data is already uploaded.
# if not refer to the original version of this notebook on how to download from web and upload them
# ds.upload(src_dir='./data/mnist', target_path='mnist', overwrite=True, show_progress=True)

## Retrieve or create a Azure Machine Learning compute
Azure Machine Learning Compute is a service for provisioning and managing clusters of Azure virtual machines for running machine learning workloads. Let's create a new Azure Machine Learning Compute in the current workspace, if it doesn't already exist. We will then run the training script on this compute target.

If we could not find the compute with the given name in the previous cell, then we will create a new compute here. This process is broken down into the following steps:

1. Create the configuration
2. Create the Azure Machine Learning compute

**This process will take a few minutes and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell.**


In [5]:
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target {}.'.format(cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                               max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

print("Azure Machine Learning Compute attached")

Found existing compute target gpu-cluster.
Azure Machine Learning Compute attached


In [6]:
est = TensorFlow(source_directory='./tf-mnist',                 
                 compute_target=compute_target,
                 entry_script='tf_mnist.py', 
                 use_gpu=True,
                 framework_version='1.13')

In [7]:
ps = RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 100),
        '--first-layer-neurons': choice(10, 50, 200, 300, 500),
        '--second-layer-neurons': choice(10, 50, 200, 500),
        '--learning-rate': loguniform(-6, -1)
    }
)

In [8]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [9]:
hd_config = HyperDriveConfig(estimator=est, 
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name='validation_acc', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=4,
                             max_concurrent_runs=4)

In [10]:
data_folder = DataReference(
    datastore=ds,
    data_reference_name="mnist_data")

In [12]:
metrics_output_name = 'metrics_output'
metrics_data = PipelineData(name='metrics_data',
                             datastore=ds)

hd_step_name='hd_step01'
hd_step = HyperDriveStep(
    name=hd_step_name,
    hyperdrive_config=hd_config,
    estimator_entry_script_arguments=['--data-folder', data_folder],
    inputs=[data_folder],
    metrics_output=metrics_data,
    allow_reuse=True)

In [13]:
conda_run_config = RunConfiguration(framework="python")
conda_run_config.target = compute_target
cd = CondaDependencies.create(pip_packages=['pandas', 'azureml-defaults'], 
                              pin_sdk_version=True)
conda_run_config.environment.python.conda_dependencies = cd

best_run_data = PipelineData('best_run_data', is_directory=True, datastore=ds)

best_run_step = PythonScriptStep(
    name='get best run',
    script_name='metrics.py',
    compute_target=compute_target,
        arguments=['--input_file', metrics_data,
               '--output_dir', best_run_data],
    inputs=[metrics_data],
    outputs=[best_run_data],
    runconfig=conda_run_config,
    source_directory=os.path.join(os.getcwd(), 'metrics'),
    allow_reuse=True
)

In [14]:
pipeline = Pipeline(workspace=ws, steps=[best_run_step])

### Run the pipeline

In [15]:
pipeline_run = exp.submit(pipeline)

Created step get best run [57cf7efa][6421d9a4-591b-4300-8556-3e28b910ea3d], (This step will run and generate new outputs)
Created step hd_step01 [2244efd1][d315f85b-b179-44ce-a2d5-f19aa6465246], (This step will run and generate new outputs)
Using data reference mnist_data for StepId [cac304ab][c34f1c52-35c4-4553-a4fd-0ffa20b750c6], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun 160a1ec9-ca00-4306-907b-e7b30ed8e859
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_crazy/runs/160a1ec9-ca00-4306-907b-e7b30ed8e859?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc


### Wait for the completion of this Pipeline run

In [16]:
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 160a1ec9-ca00-4306-907b-e7b30ed8e859
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_crazy/runs/160a1ec9-ca00-4306-907b-e7b30ed8e859?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc
PipelineRun Status: Running


StepRunId: 50cc7c9c-adff-4274-ace9-032ea4d2cebc
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_crazy/runs/50cc7c9c-adff-4274-ace9-032ea4d2cebc?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc
StepRun( hd_step01 ) Status: NotStarted
StepRun( hd_step01 ) Status: Running

StepRun(hd_step01) Execution Summary
StepRun( hd_step01 ) Status: Finished
{'runId': '50cc7c9c-adff-4274-ace9-032ea4d2cebc', 'status': 'Completed', 'startTimeUtc': '2020-04-20T17:27:41.560383Z', 'endTimeUtc': '2020-04-20T17:47:01.664323Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'Cont

'Finished'

In [17]:
pipeline_run2 = exp.submit(pipeline)

Submitted PipelineRun 0992d2e5-797a-4122-bbeb-523a524d07b0
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_crazy/runs/0992d2e5-797a-4122-bbeb-523a524d07b0?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc


In [18]:
pipeline_run2.wait_for_completion(show_output=True)

PipelineRunId: 0992d2e5-797a-4122-bbeb-523a524d07b0
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_crazy/runs/0992d2e5-797a-4122-bbeb-523a524d07b0?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 7424807e-5660-4817-8686-28de6ad56ec1
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_crazy/runs/7424807e-5660-4817-8686-28de6ad56ec1?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc

StepRun(get best run) Execution Summary
StepRun( get best run ) Status: Finished
{'runId': '7424807e-5660-4817-8686-28de6ad56ec1', 'target': 'gpu-cluster', 'status': 'Completed', 'startTimeUtc': '2020-04-20T17:49:24.924566Z', 'endTimeUtc': '2020-04-20T17:49:24.997065Z', 'properties': {'azureml.reusedrunid': 'cadc8048-9fde-4d94-9f48-1f77ec54ccc0

'Finished'

In [19]:
pipeline3 = Pipeline(workspace=ws, steps=[best_run_step])

In [20]:
pipeline_run3 = exp.submit(pipeline3)

Created step get best run [c5fa096d][6421d9a4-591b-4300-8556-3e28b910ea3d], (This step is eligible to reuse a previous run's output)
Created step hd_step01 [3ffca693][d315f85b-b179-44ce-a2d5-f19aa6465246], (This step is eligible to reuse a previous run's output)
Using data reference mnist_data for StepId [74583c11][c34f1c52-35c4-4553-a4fd-0ffa20b750c6], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun 65680d46-9a43-4fdd-8835-5ea0df82da77
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_crazy/runs/65680d46-9a43-4fdd-8835-5ea0df82da77?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc


In [21]:
pipeline_run3.wait_for_completion(show_output=True)

PipelineRunId: 65680d46-9a43-4fdd-8835-5ea0df82da77
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_crazy/runs/65680d46-9a43-4fdd-8835-5ea0df82da77?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 6a2c3fbd-2ffa-443f-bee0-4aebbb43b3f4
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_crazy/runs/6a2c3fbd-2ffa-443f-bee0-4aebbb43b3f4?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc

StepRun(get best run) Execution Summary
StepRun( get best run ) Status: Finished
{'runId': '6a2c3fbd-2ffa-443f-bee0-4aebbb43b3f4', 'target': 'gpu-cluster', 'status': 'Completed', 'startTimeUtc': '2020-04-20T17:49:39.572822Z', 'endTimeUtc': '2020-04-20T17:49:39.817025Z', 'properties': {'azureml.reusedrunid': 'cadc8048-9fde-4d94-9f48-1f77ec54ccc0

'Finished'